In [1]:
!pip3 install opencv-python

<p>cv2.HoughCircles(image, method, dp, minDist)</p>
<ul>Where:
    <li>Image is the image file converted to grey scale.</li>
    <li>Method is the algorithm used to detct the circles.</li>
    <li>Dp is the inverse ratio of the accumulator resolution to the image resolution.</li>
    <li>minDist is the Minimum distance between the center coordinates of detected circles.</li></ul>

## The aim is to learn detecting and calculating the amount of coins in the photo.

### First try

In [18]:
import cv2
import numpy as np

image = cv2.imread('my_coins1.jpg')

output = image.copy()

img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find circles
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1.3, 100)

# If some circle is found
if circles is not None:
    
   # Get the (x, y, r) as integers
    circles = np.round(circles[0, :]).astype("int")
    print(circles)
    
   # loop over the circles
    for (x, y, r) in circles:
        cv2.circle(output, (x, y), r, (0, 255, 0), 2)
        
# show the output image
cv2.imshow("circle", output)
cv2.waitKey(0)

[[457 111  90]
 [236 131  70]
 [738 601  70]
 [510 522  69]
 [951 479  82]
 [287 257  56]
 [821 275  63]]


-1

### Steps:
<ol> <li> Original Image --> Gaussian Blur --> Determining Edges </li>
    <li> Generate the reference circles (first two loops)</li>
    <li> Identify Coins </li></ol>
    

In [4]:
# 1. Blurring
import cv2
import numpy

# read image
src = cv2.imread('my_coins1.jpg', cv2.IMREAD_UNCHANGED)

# apply guassian blur on src image
dst = cv2.GaussianBlur(src,(1,1), cv2.BORDER_DEFAULT)  #choosing how much to blur

# display input and output image
cv2.imshow("Gaussian Blur", dst)
save_me = cv2.imwrite('blurred_coins1.jpg', dst)  #save your cv2 image

cv2.waitKey(0) # closes when key is pressed
cv2.destroyAllWindows() # destroys the window showing image

In [5]:
# 2. Determine Edges
import cv2
import numpy as np
import matplotlib.pyplot as plt


image = cv2.imread('blurred_coins1.jpg')

output = image.copy()

img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find circles
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1.3, 100)

# If some circle is found
if circles is not None:
    
   # Get the (x, y, r) as integers
    circles = np.round(circles[0, :]).astype("int")
    print(circles)
    
   # loop over the circles
    for (x, y, r) in circles:
        cv2.circle(output, (x, y), r, (0, 255, 0), 2)
        
# show the output image
cv2.imshow("circle", output)
cv2.waitKey(0)
cv2.destroyAllWindows() # destroys the window showing image

[[262 244 180]
 [288 146 146]
 [314 335 118]
 [ 71 427 329]
 [151 242  93]
 [362 242 102]
 [  7 272 236]
 [298 454 127]
 [ 76 135  89]
 [358  72  53]
 [  3   7 217]
 [203  55  38]
 [196 345  44]
 [103  37  34]]


### Hough Circles
<li><b>gray:</b> Input image (grayscale).</li>
<li><b>circles:</b> A vector that stores sets of 3 values: xc,yc,r for each detected circle.</li>
<li><b>HOUGH_GRADIENT:</b> Define the detection method. Currently this is the only one available in OpenCV.</li>
<li><b>dp = 1:</b> The inverse ratio of resolution.</li>
<li><b>min_dist = gray.rows/16:</b> Minimum distance between detected centers.</li>
<li><b>param_1 = 200:</b> Upper threshold for the internal Canny edge detector.</li>
<li><b>param_2 = 100*:</b> Threshold for center detection.</li>
<li><b>min_radius = 0:</b> Minimum radius to be detected. If unknown, put zero as default.</li>
<li><b>max_radius = 0:</b> Maximum radius to be detected. If unknown, put zero as default.</li>

In [1]:
#Using Tina's source code
"""
Coin recognition, real life application
task: calculate the value of coins on picture
"""

import cv2    # import essential libraries
import numpy as np

def detect_coins():
    coins = cv2.imread('my_coins1.jpg', 1)   #import the photo

    gray = cv2.cvtColor(coins, cv2.COLOR_BGR2GRAY)   # filter to make it easier
    img = cv2.medianBlur(gray, 7)     # light blur
    circles = cv2.HoughCircles(      # detecting the circles
        img,  # source image
        cv2.HOUGH_GRADIENT,  # type of detection
        1,   #inverse ratio of revolution
        50,   # minimum distance between detected circles
        param1=100,   
        param2=50,
        minRadius=10,  # minimal radius
        maxRadius=380,  # max radius
    )

    coins_copy = coins.copy()   # copy the image coins


    for detected_circle in circles[0]:   # for each detected circle
        x_coor, y_coor, detected_radius = detected_circle    # their coordinates and radius
        coins_detected = cv2.circle(coins_copy, (int(x_coor), int(y_coor)), int(detected_radius), (0, 255, 0), 4,)

    cv2.imwrite("my_coins_Tina.jpg", coins_detected)

    return circles

def calculate_amount():
    tenges = {
        "1 Tenge": {
            "value": 1,     
            "radius": 20,     # 15 (diameter)
            "ratio": 1,
            "count": 0,
        },
        "5 Tenge": {
            "value": 5,
            "radius": 21.5,  # 17,27 (diameter)
            "ratio": 1.1,
            "count": 0,
        },
        "20 Tenge": {
            "value": 20,   
            "radius": 24.5,   # 18,27 (diameter)
            "ratio": 1.17,
            "count": 0,
        },
        "50 Tenge": {
            "value": 50,
            "radius": 30,   # 23 (diameter)
            "ratio": 1.45,
            "count": 0,
        },
        "10 Tenge": {
            "value": 10,  
            "radius": 27.5,    # 19,56 (diameter)
            "ratio": 1.3,
            "count": 0,
        },
}

    circles = detect_coins()
    radius = []
    coordinates = []

    for detected_circle in circles[0]:
        x_coor, y_coor, detected_radius = detected_circle
        radius.append(detected_radius)
        coordinates.append([x_coor, y_coor])
        #print(detected_radius)

    smallest = min(radius)
    tolerance = 0.0375
    total_amount = 0

    coins_circled = cv2.imread('my_coins_Tina.jpg', 1)
    font = cv2.FONT_HERSHEY_SIMPLEX

    for coin in circles[0]:
        ratio_to_check = coin[2] / smallest
        coor_x = coin[0]
        coor_y = coin[1]
        for tenge in tenges:
            value = tenges[tenge]['value']
            if abs(ratio_to_check - tenges[tenge]['ratio']) <= tolerance:
                tenges[tenge]['count'] += 1
                total_amount += tenges[tenge]['value']
                cv2.putText(coins_circled, str(value), (int(coor_x), int(coor_y)), font, 1.5, (0, 0, 0), 4)   #putting text

    print(f"The total amount is {total_amount} Tenge")

    for tenge in tenges:
        pieces = tenges[tenge]['count']
        print(f"{tenge} = {pieces}x")


    cv2.imwrite("my_coins_Tina.jpg", coins_circled)



if __name__ == "__main__":
    calculate_amount()
    img = cv2.imread("my_coins_Tina.jpg", cv2.IMREAD_COLOR)
    cv2.imshow("my_calculated_coins", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


The total amount is 81 Tenge
1 Tenge = 1x
5 Tenge = 2x
20 Tenge = 1x
50 Tenge = 1x
10 Tenge = 0x


### Course Try - understandable

In [1]:
# My Own Try

# Identifying coins by their brightness and radii
import cv2
import numpy as np

def av_pix(my_img,circles,size):   # brightness
    av_value = []
    for coords in circles[0,:]:
        col = np.mean(my_img[coords[1]-size:coords[1]+size,coords[0]-size:coords[0]+size])
        #print(img[coords[1]-size:coords[1]+size,coords[0]-size:coords[0]+size])
        av_value.append(col)
    return av_value   


def get_radius(circles):   # radii identifying
    radius = []
    for coords in circles[0,:]:
        radius.append(coords[2])    
    return radius


my_img = cv2.imread('capstone-coins1.png', cv2.IMREAD_GRAYSCALE)    # turning the photo gray
original_img = cv2.imread('capstone-coins1.png', 1)   # just original photo
my_img = cv2.GaussianBlur(my_img, (3,3),0)    # making blur


#Hough Circle Detection
circles = cv2.HoughCircles(my_img, cv2.HOUGH_GRADIENT, 0.9, 120, param1=50, param2=27, minRadius=60, maxRadius=120)
# print(circles)

circles = np.uint16(np.around(circles))  # round floats to integers 

count = 0
for i in circles[0,:]:  # for each circle
    #draw the outer circle
    cv2.circle(my_img, (i[0],i[1]),i[2],(0,255,0),2)
    
    #draw the center of the circle
    cv2.circle(my_img, (i[0],i[1]),2,(0,255,0),3)
    cv2.putText(my_img, str(count),(i[0],i[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2)
    
    count+=1
    

radii = get_radius(circles)
print(radii, " - radii")

bright_values = av_pix(my_img,circles,20)
print(bright_values, " - brightness")

values = []
for a,b in zip(bright_values,radii):    # classifying coins by their brightness and radii
    if a > 150 and b > 110:
        values.append(10)
    elif a > 150 and b <= 110:
        values.append(5)
    elif a < 150 and b > 110:
        values.append(2)
    elif a < 150 and b < 110:
        values.append(1)        
print(values)           
count_2 = 0

# for i in circles[0,:]:
#     cv2.putText(my_img, str(values[count_2]) + 'p',(i[0],i[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2)  # somewhere mistake - list out of index
#     count_2 += 1
    
# cv2.putText(my_img, 'ESTIMATED TOTAL VALUE: ' + str(sum(values)) + 'p', (200,100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, 255)
print()
print(f'ESTIMATED TOTAL VALUE: {sum(values)} tenge')

# cv2.imshow('Photos of coins', my_img)     # show the photos
# cv2.waitKey(0)     # when key pressed
# cv2.destroyAllWindows()  # close all windows


[89, 118, 103, 94, 119, 103, 99, 110, 119, 72]  - radii
[157.105625, 185.263125, 71.174375, 123.93875, 91.0225, 73.185, 95.730625, 145.915, 81.786875, 91.119375]  - brightness
[5, 10, 1, 1, 2, 1, 1, 2, 1]

ESTIMATED TOTAL VALUE: 24 tenge
